In [1]:
import numpy as np
from keras.datasets import cifar10
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords
import spacy

In [2]:
# Load the dataset (replace 'spam_dataset.csv' with the actual filename)
economy = pd.read_csv("/content/Economy.csv")
sports=pd.read_csv('/content/Sports.csv')
art=pd.read_csv('/content/Art.csv')


In [3]:
economy

,Unnamed: 0,Article Title,Category
0,0,"\n\nChinese demand, Mideast turmoil boost ship...",economy
1,1,\n\nEgypt government to issue LE1 bn in zero c...,economy
2,2,\n\nFirst Egyptian Samsung factory starts prod...,economy
3,3,\n\nUPDATED: Egyptian and Arab stocks soar as ...,economy
4,4,\n\nGreek high school teachers to strike over ...,economy
...,...,...,...
9995,9975,Egypt imposes three-month export ban on cookin...,economy
9996,9976,"Egypt bans export of wheat, other staples for ...",economy
9997,9977,"Egypt’s annual inflation hit 10% in February, ...",economy
9998,9978,INTERVIEW: EBRD supports Egypt’s green transfo...,economy


In [4]:
sports

,Unnamed: 0,Article Title,Category
0,0,\n\nFans in shock after Leicester City helicop...,sports
1,1,\n\nLeicester City football club owner was on ...,sports
2,2,\n\nLeicester City football club owner's helic...,sports
3,3,\n\nRonaldo scores twice as Juve overcome frig...,sports
4,4,\n\nKlopp hails Liverpool's '20-point win' ove...,sports
...,...,...,...
9975,9955,Germany Bundesliga results & scorers (21st mat...,sports
9976,9956,Barcelona go fourth after winning six-goal thr...,sports
9977,9957,Dortmund humiliated by Leverkusen in Haaland's...,sports
9978,9958,"Back from injury, 1st Liverpool goal for teen ...",sports


In [5]:
art

,Unnamed: 0,Article Title,Category
0,0,\n\nWorks by Picasso and others stolen in Spai...,Art
1,1,\n\nSalwa and Maged at the Art Corner Gallery ...,Art
2,2,\n\nA tribute to Abdalla Gohar\n\n\n\nOn 1 Dec...,Art
3,3,"\n\nRaw, untamed, and compelling: ‘Flesh’ by S...",Art
4,4,\n\nFrom water to mosaics\n\n\n\nThe Gezira Ce...,Art
...,...,...,...
10452,10452,Netflix’s first Arabic film production remains...,Art
10453,10453,Disney+ to launch in Egypt this summer\n\n\n\n...,Art
10454,10454,China gives 'Fight Club' new ending where auth...,Art
10455,10455,Egypt’s Actors’ Syndicate defends Mona Zaki as...,Art


In [6]:
# Concatenate the three datasets along the rows axis (vertically)
final_df = pd.concat([economy, sports, art])

# Write the concatenated DataFrame to a CSV file
final_df.to_csv('concatenated_dataset.csv', index=False)

In [7]:
final_df

,Unnamed: 0,Article Title,Category
0,0,"\n\nChinese demand, Mideast turmoil boost ship...",economy
1,1,\n\nEgypt government to issue LE1 bn in zero c...,economy
2,2,\n\nFirst Egyptian Samsung factory starts prod...,economy
3,3,\n\nUPDATED: Egyptian and Arab stocks soar as ...,economy
4,4,\n\nGreek high school teachers to strike over ...,economy
...,...,...,...
10452,10452,Netflix’s first Arabic film production remains...,Art
10453,10453,Disney+ to launch in Egypt this summer\n\n\n\n...,Art
10454,10454,China gives 'Fight Club' new ending where auth...,Art
10455,10455,Egypt’s Actors’ Syndicate defends Mona Zaki as...,Art


In [8]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30437 entries, 0 to 10456
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     30437 non-null  int64 
 1   Article Title  30437 non-null  object
 2   Category       30437 non-null  object
dtypes: int64(1), object(2)
memory usage: 951.2+ KB


In [9]:
final_df.drop('Unnamed: 0', axis = 1, inplace=True)

In [10]:
final_df


,Article Title,Category
0,"\n\nChinese demand, Mideast turmoil boost ship...",economy
1,\n\nEgypt government to issue LE1 bn in zero c...,economy
2,\n\nFirst Egyptian Samsung factory starts prod...,economy
3,\n\nUPDATED: Egyptian and Arab stocks soar as ...,economy
4,\n\nGreek high school teachers to strike over ...,economy
...,...,...
10452,Netflix’s first Arabic film production remains...,Art
10453,Disney+ to launch in Egypt this summer\n\n\n\n...,Art
10454,China gives 'Fight Club' new ending where auth...,Art
10455,Egypt’s Actors’ Syndicate defends Mona Zaki as...,Art


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
final_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

,Article Title,Category
0,"Chinese demand, Mideast turmoil boost shipping...",economy
1,Egypt government to issue LE1 bn in zero coupo...,economy
2,First Egyptian Samsung factory starts producti...,economy
3,UPDATED: Egyptian and Arab stocks soar as Russ...,economy
4,Greek high school teachers to strike over job ...,economy
...,...,...
10452,Netflix’s first Arabic film production remains...,Art
10453,Disney+ to launch in Egypt this summer\n\n\n\n...,Art
10454,China gives 'Fight Club' new ending where auth...,Art
10455,Egypt’s Actors’ Syndicate defends Mona Zaki as...,Art


In [13]:

# Load English stopwords
stop_words = set(stopwords.words('english'))

# Load spaCy English model for lemmatization
nlp = spacy.load('en_core_web_sm')

# Function to remove stopwords and perform lemmatization on text
def clean_text(text):
    # Tokenize the text
    doc = nlp(text)
    # Lemmatize tokens and remove stopwords
    cleaned_tokens = [token.lemma_ for token in doc if token.text.lower() not in stop_words]
    # Join the tokens back into a string
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text


# Clean the 'text_column' using the clean_text function
    final_df['cleaned_text'] = final_df['Article Title'].apply(clean_text)



In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()
tfidf_embeddings = tfidf_vectorizer.fit_transform(final_df['Article Title'])

# Now tfidf_embeddings contains the TF-IDF embeddings of the text data

In [15]:
tfidf_embeddings

<30437x27137 sparse matrix of type '<class 'numpy.float64'>'
	with 549663 stored elements in Compressed Sparse Row format>

In [17]:
from gensim.models import FastText

sentences = [text.split() for text in final_df['Article Title']]
fasttext_model = FastText(sentences, vector_size=1000, window=5, min_count=1, workers=4, sg=1)
fasttext_embeddings = [fasttext_model.wv.get_vector(" ".join(sentence)) for sentence in sentences]

# Now fasttext_embeddings contains the FastText embeddings of the text data


In [19]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument


tagged_data = [TaggedDocument(words=text.split(), tags=[i]) for i, text in enumerate(final_df['Article Title'])]
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, workers=4, epochs=20)
doc2vec_model.build_vocab(tagged_data)
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
doc2vec_embeddings = [doc2vec_model.infer_vector(text.split()) for text in final_df['Article Title']]

# Now doc2vec_embeddings contains the Doc2Vec embeddings of the text data


In [16]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


X = tfidf_embeddings  # Use the TF-IDF embeddings for X
# X = fasttext_embeddings  # Use the FastText embeddings for X
# X = sentence_embeddings  # Use the Sentence Transformer embeddings for X
y = final_df['Category']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model = SVC(kernel='linear')

# Fit the model on the training data
svm_model.fit(X_train, y_train)

# Predict the target variable on the testing data
y_pred = svm_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.989323258869908


In [18]:

#X = TF-IDF embeddings  # Use the TF-IDF embeddings for X
X2 = fasttext_embeddings  # Use the FastText embeddings for X
# X = sentence_embeddings  # Use the Sentence Transformer embeddings for X
#y2 = final_df['Category']

# Split the data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model2 = SVC(kernel='linear')

# Fit the model on the training data
svm_model2.fit(X_train2, y_train2)

# Predict the target variable on the testing data
y_pred2 = svm_model2.predict(X_test2)

# Calculate the accuracy of the model
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy:", accuracy2)

Accuracy: 0.9397174770039421


In [44]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


sentences = [word_tokenize(text.lower()) for text in final_df['Article Title']]

# Train the Word2Vec model
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Convert sentences to Word2Vec embeddings
embeddings = []
for sentence in sentences:
    sentence_embedding = []
    for word in sentence:
        if word in word2vec_model.wv:
            sentence_embedding.append(word2vec_model.wv[word])
    if sentence_embedding:
        embeddings.append(sum(sentence_embedding) / len(sentence_embedding))
    else:
        embeddings.append([0] * 100)  # If no word in the sentence is present in the Word2Vec model, use zero vector


X_word = embeddings
y = final_df['Category']

# Split the data into training and testing sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X_word, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model3 = SVC(kernel='linear')

# Fit the model on the training data
svm_model3.fit(X_train3, y_train3)

# Predict the target variable on the testing data
y_pred3 = svm_model3.predict(X_test3)

# Calculate the accuracy of the model
accuracy3 = accuracy_score(y_test3, y_pred3)
print("Accuracy:", accuracy3)



Accuracy: 0.922634691195795


In [20]:
#X = TF-IDF embeddings  # Use the TF-IDF embeddings for X
#X3 = word2vec_embeddings  # Use the FastText embeddings for X
X4 = doc2vec_embeddings  # Use the Sentence Transformer embeddings for X
#y2 = final_df['Category']

# Split the data into training and testing sets
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model4 = SVC(kernel='linear')

# Fit the model on the training data
svm_model4.fit(X_train4, y_train4)

# Predict the target variable on the testing data
y_pred4 = svm_model4.predict(X_test4)

# Calculate the accuracy of the model
accuracy4 = accuracy_score(y_test4, y_pred4)
print("Accuracy:", accuracy4)

Accuracy: 0.8157030223390276


In [ ]:
#X = TF-IDF embeddings  # Use the TF-IDF embeddings for X
#X3 = word2vec_embeddings  # Use the FastText embeddings for X
X5 = sentence_embeddings  # Use the Sentence Transformer embeddings for X
#y2 = final_df['Category']

# Split the data into training and testing sets
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y, test_size=0.2, random_state=42)

# Initialize the SVM model
svm_model5 = SVC(kernel='linear')

# Fit the model on the training data
svm_model5.fit(X_train5, y_train5)

# Predict the target variable on the testing data
y_pred5 = svm_model5.predict(X_test5)

# Calculate the accuracy of the model
accuracy5 = accuracy_score(y_test5, y_pred5)
print("Accuracy:", accuracy5)

In [27]:
!pip install transformers


In [28]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB ? eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 375.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 679.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━